In [ ]:
import glob
import os
import sys
import numpy as np
import cv2

In [ ]:
# import tensorflow
import time

In [ ]:
import ultralytics
from ultralytics import YOLO
yolo=YOLO("yolov8n.pt")


In [ ]:
from PIL import Image

In [ ]:
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


In [ ]:
import carla

In [ ]:
def process(image):
    i=np.array(image.raw_data)
    i2=i.reshape((600,600,4))
    i3=i2[:,:,:3]
   # cv2.imshow("",i3)
    # cv2.waitKey(1)
    normi=(i3/255.0)
    opi=cv2.cvtColor((normi*255).astype(np.uint8),cv2.COLOR_RGB2BGR)
    return opi

In [ ]:
def cm_callback(image):
    img=np.array(image.raw_data).reshape((600,600,4))
    img=cv2.cvtColor(img,cv2.COLOR_BGRA2BGR)
    result=yolo(img)[0]
    for box in result.boxes:
        x1,y1,x2,y2=box.xyxy[0]
        x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        label=result.names[int(box.cls[0])]
        cv2.putText(img,label,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(36,255,12),2)
    # img.save_to_disk('data_carla/%06d.png' % img.frame,carla.ColorConverter.Raw)
    pi=Image.fromarray(img)
    # pi.save_to_disk('data_carla/%06d.png' % pi.frame,carla.ColorConverter.Raw)
    frame_id=str(time.time())
    file_p=os.path.join('data_carla',f'dete_obj_{frame_id}.png')
    pi.save(file_p)
    cv2.waitKey(0)

In [ ]:
actor_list=[]


In [ ]:

client=carla.Client('localhost',2000)
client.set_timeout(2000.0)
world=client.get_world()
bpl=world.get_blueprint_library()
bp=bpl.filter('model3')[0]
print(bp)
spp=world.get_map().get_spawn_points()
sp=spp[10]
vl=world.spawn_actor(bp,sp)
actor_list.append(vl)


In [ ]:
#vl.set_autopilot(True)

In [ ]:
#vehicle controle whitout autopilote
vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))

In [ ]:
bp2=bpl.find('sensor.camera.rgb')
bp2.set_attribute("image_size_x",f'{600}')
bp2.set_attribute("image_size_y",f'{600}')
bp2.set_attribute('fov','110')
#position of the sensor
ssp=carla.Transform(carla.Location(x=2.5,z=0.7))
sn=world.spawn_actor(bp2,ssp,attach_to=vl)
actor_list.append(sn)

In [ ]:
#vehicle controle without autopilote
vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))
sn.listen(cm_callback)

In [ ]:
#vl.destroy()
#sn.destroy()
for actor in actor_list:
    actor.destroy()